## EJERCICIO 3


**Enunciado**: Usa la clase PairWiseAligner de Biopython para alinear secuencias
de aminoácidos (proteínas) y obtener la puntuación y el alineamiento óptimo
en estos dos escenarios.


a) Generar las secuencias aleatoriamente (de forma automática no manual).


b) Obtener las secuencias a partir de ficheros obtenidos de bases de
datos biológicas (como Uniprot). Explicar cómo se obtuvieron.
Prueba a usar diferentes matrices de sustitución, como BLOSUM62, PAM250 o
una matriz personalizada. Compara los resultados obtenidos con diferentes
matrices e investiga las ventajas y desventajas de cada una.